In [5]:
from pathlib import Path
from pprint import pprint

def list_daily_flow_files(base_dir: Path | None = None):
    """
    1. 取得 '每日各站進出站人數' 資料夾
    2. 取得其中所有 CSV 檔案的絕對路徑
    3. 排除 manifest.csv 與 schema.csv
    """
    if base_dir is None:
        base_dir = Path.cwd()  # 與 lesson19_2.ipynb 做法一致
    target_dir = base_dir / "每日各站進出站人數"
    if not target_dir.is_dir():
        raise FileNotFoundError(f"找不到資料夾: {target_dir}")

    excluded = {"manifest.csv", "schema.csv"}
    csv_paths = sorted(p.resolve()
        for p in target_dir.glob("*.csv")
        if p.name not in excluded
    )
    return target_dir.resolve(), csv_paths

folder_abs_path, file_abs_paths = list_daily_flow_files()

print("資料夾絕對路徑:")
print(folder_abs_path)
print("\n檔案絕對路徑 (已排除 manifest.csv, schema.csv):")
pprint([str(p) for p in file_abs_paths])

資料夾絕對路徑:
C:\Users\User\Documents\Github\LWY\lesson19\每日各站進出站人數

檔案絕對路徑 (已排除 manifest.csv, schema.csv):
['C:\\Users\\User\\Documents\\Github\\LWY\\lesson19\\每日各站進出站人數\\每日各站進出站人數20190423-20191231.csv',
 'C:\\Users\\User\\Documents\\Github\\LWY\\lesson19\\每日各站進出站人數\\每日各站進出站人數2020.csv',
 'C:\\Users\\User\\Documents\\Github\\LWY\\lesson19\\每日各站進出站人數\\每日各站進出站人數2021.csv',
 'C:\\Users\\User\\Documents\\Github\\LWY\\lesson19\\每日各站進出站人數\\每日各站進出站人數2022.csv',
 'C:\\Users\\User\\Documents\\Github\\LWY\\lesson19\\每日各站進出站人數\\每日各站進出站人數2023.csv']
